<a href="https://colab.research.google.com/github/LazarevaL/AI_hack_lipetsk/blob/main/AI_hack_lipetsk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузим нужные библиотеки

In [8]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm
from os import listdir 

from sklearn.model_selection import KFold

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 26.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.3 MB/s 
     |████████████████████████████████| 163 kB 47.9 MB/s 


In [4]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import warnings

from transformers import AutoTokenizer, AutoModel
import torch
import re

warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive 
drive.mount('/disk')

Mounted at /disk


In [77]:
train = pd.read_csv("/disk/MyDrive/Colab_Notebooks/train_dataset_train_1/train.csv")

In [78]:
train

,id,description,object_img
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244
...,...,...,...
2093,1464,Фотонегатив пленочный. Сцена из спектакля «Жен...,836
2094,1297,Монета ДМ/Н-329 ПОКМ-20597/124 серебро; штампо...,61
2095,1507,Фотография. Уральский семинар секретарей комит...,1290
2096,1863,ПЗС. Подвеска шумящая коньковая на пронизи ПК...,1949


In [76]:
sample = pd.read_csv("/disk/MyDrive/Colab_Notebooks/train_dataset_train_1/sample_solution.csv")

## Добавим новых признаков

In [4]:
def defineBaseDesc(seq):
    
    t = seq.split(".")[0]
    if len(t.split(" ")) > 2:
        t = t.split(" ")[0]
        if "," in t:
            return t.split(",")[0]
        else:
            return t
    else:
        if "Этюд" in t:
            t = "Этюд"
        return t

In [5]:
def desc1CheckUp(val):
    
    val = val.split(" ")
    if len(val) > 1:
        if len(val[1]) < 3:
            return val[0]
        elif "," in val[0]:
            return val[0].split(",")[0]
        else:
            return " ".join(val)
    else:
        if "," in val[0]:
            return val[0].split(",")[0]
        elif val[0] in ["Двое", "«Дружки»", "Боронование", "Типы", "Перевозка", "Группа", "Четыре", "Лошадь", 
                     "Крестьяне", "Крестьянки", "Подростки", "Деревянная", "Старушки", "Два"]:
            return "Изображение"
        elif val[0] == "Казенная":
            return "Фрагмент винтовки"
        else:
            return val[0]

In [17]:
def getEmbeddings(data, languageModel, column, maxLen=128):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    tokenizer = AutoTokenizer.from_pretrained(languageModel)
    model = AutoModel.from_pretrained(languageModel).to(device)
    
    colName, featues  = languageModel.split("/")[1], []
    for text in tqdm(data[column]):
        encoded = tokenizer([text], padding="max_length", add_special_tokens=True,
                            truncation=True, max_length=maxLen, return_tensors='pt')

        with torch.no_grad():
            output = model(input_ids=encoded['input_ids'].to(device))
            featues.append(output[0].detach().to("cpu").sum(dim=1).numpy()[0])

    res = pd.DataFrame(columns = [f"{colName}_f{i}" for i in range(len(featues[0]))], data=featues)
    return res

In [18]:
def validateModel(model, data, target, fitParams, nFold=5, KFold_randomState=42):

    kFold = KFold(n_splits=nFold, shuffle=True, random_state=KFold_randomState)

    X = data
    y = target
   
    print(f"[X]: {X.shape} \n[y]: {y.shape}")

    cv = []
    for i, idx in kFold.split(X):
    
        X_train, X_valid = X.loc[i], X.loc[idx]
        y_train, y_valid = y.loc[i], y.loc[idx]

        try:
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], **fitParams)
        except:
            model.fit(X_train, y_train)
    
        r2 = r2_score(y_valid, model.predict(X_valid))
        cv.append(r2)
        print(f"[{i}]: {r2}")

    print(f"[mean]: {np.mean(cv)}")

In [ ]:
train["desc"]= train["desc"].str.strip()
train["desc_len"] = train["desc"].str.len()

train["desc_1"] = train["desc"].apply(defineBaseDesc)
train["desc_1"] = train["desc_1"].apply(desc1CheckUp)
train = pd.get_dummies(train, columns=["desc_1"], dtype='int8')

In [ ]:
# LANG MODELS

langModels = ["cointegrated/rubert-tiny2", "cointegrated/LaBSE-en-ru", "sberbank-ai/ruBert-base"]
for model in langModels:
    train = train.join(getEmbeddings(train, model, "desc")) 
train.drop(["desc"], axis=1, inplace=True)

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2098/2098 [16:12<00:00,  2.16it/s]


Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2098/2098 [15:55<00:00,  2.20it/s]


In [ ]:
train.sample(5)

,object_img,len_description,object,desc_len,path,desc_1_Авиабомба,desc_1_Алебарда,desc_1_Альбом,desc_1_Амбары,desc_1_Бердыш,...,ruBert-base_f758,ruBert-base_f759,ruBert-base_f760,ruBert-base_f761,ruBert-base_f762,ruBert-base_f763,ruBert-base_f764,ruBert-base_f765,ruBert-base_f766,ruBert-base_f767
1507,842,160,80,160,./train/842.png,0,0,0,0,0,...,-82.673958,-48.242519,-53.796410,38.519474,-35.959885,-57.166260,14.577268,18.701813,-6.590319,1.603873
1079,1086,197,80,197,./train/1086.png,0,0,0,0,0,...,-75.401276,-33.827969,-34.361000,43.265472,-32.811600,-54.760849,20.999050,16.148146,-6.264384,-6.472845
729,2105,372,42,372,./train/2105.png,0,0,0,0,0,...,-46.878670,-12.240154,-36.216782,30.015450,-7.142320,-45.512997,25.487656,38.135532,29.225189,8.201272
1058,1801,218,21,218,./train/1801.png,0,0,0,0,0,...,-42.093842,-21.278000,-31.539230,46.422180,-15.291065,-47.048420,31.460136,33.848305,8.071037,4.913805
431,2704,191,12,191,./train/2704.png,0,0,0,0,0,...,-61.914852,-12.133757,-32.909458,27.511858,-29.699509,-71.797127,1.807203,48.674286,-1.531954,11.925743


In [ ]:
#train.to_csv('/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/train_tokens.csv', index=False )

In [120]:
#train = pd.read_csv("/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/train_tokens.csv")

In [8]:
train.corrwith(train.object_img)

object_img          1.000000
len_description     0.074419
object             -0.505831
desc_len            0.077225
desc_1_Авиабомба    0.021373
                      ...   
ruBert-base_f763   -0.004914
ruBert-base_f764   -0.159762
ruBert-base_f765    0.170366
ruBert-base_f766    0.061237
ruBert-base_f767   -0.134507
Length: 1926, dtype: float64

## Выделим выборки

In [121]:
X = train.drop(["object_img", 'desc_len', 'path'], axis = 1)
y = train['object_img']

In [81]:
X

,len_description,object,desc_1_Авиабомба,desc_1_Алебарда,desc_1_Альбом,desc_1_Амбары,desc_1_Бердыш,desc_1_Бомба,desc_1_Винтовка,desc_1_Гербарий,...,ruBert-base_f758,ruBert-base_f759,ruBert-base_f760,ruBert-base_f761,ruBert-base_f762,ruBert-base_f763,ruBert-base_f764,ruBert-base_f765,ruBert-base_f766,ruBert-base_f767
0,84,78,0,0,0,0,0,0,0,0,...,-102.432270,-36.818980,-70.645260,57.091904,-46.113720,-57.591900,10.264516,37.627130,-28.816969,-7.142577
1,145,80,0,0,0,0,0,0,0,0,...,-90.702610,-43.682190,-60.133286,45.315044,-34.549076,-50.053410,19.595024,25.940004,-20.161310,-6.931845
2,314,21,0,0,0,0,0,0,0,0,...,-18.724703,-31.980110,-42.833780,40.749800,-24.217040,-23.049948,30.429090,16.681332,28.066965,8.125870
3,287,80,0,0,0,0,0,0,0,0,...,-37.308113,-26.900326,-40.067127,28.872707,-13.920437,-52.614670,34.357285,28.070728,5.852749,16.395205
4,322,80,0,0,0,0,0,0,0,0,...,-40.167763,-15.792139,-34.886337,22.984747,-17.400011,-43.129990,18.395967,12.017011,7.932607,23.269210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,163,80,0,0,0,0,0,0,0,0,...,-87.418260,-40.222084,-48.860264,37.969074,-33.245030,-52.774483,15.102554,15.934387,-10.886412,2.127125
2094,622,36,0,0,0,0,0,0,0,0,...,-4.114290,-13.697747,-32.238464,33.945038,-24.435492,-6.654069,29.415108,2.762140,62.421017,45.586338
2095,157,78,0,0,0,0,0,0,0,0,...,-87.319275,-34.156677,-48.099720,52.467926,-41.151752,-57.515945,14.250944,30.867792,-17.352253,1.584009
2096,533,42,0,0,0,0,0,0,0,0,...,-37.114796,2.164124,-1.210188,39.420240,-6.669504,-16.544752,27.007970,9.324976,51.074350,37.240320


In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Работа с моделью 

In [83]:
import numpy as np

In [57]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [58]:
from catboost import CatBoostRegressor

In [123]:
model = CatBoostRegressor(iterations=1000, random_seed=42, task_type="CPU", eval_metric="R2",
                           use_best_model=False)
validateModel(model, X, y, fitParams={"early_stopping_rounds": 100, "verbose": 0},nFold=5, KFold_randomState=42)

[X]: (2098, 1924) 
[y]: (2098,)
[[   0    1    2 ... 2095 2096 2097]]: 0.9249840132152848
[[   0    1    3 ... 2095 2096 2097]]: 0.9362839664857501
[[   0    1    2 ... 2094 2096 2097]]: 0.9128846864881234
[[   1    2    3 ... 2092 2095 2096]]: 0.9258310018548903
[[   0    2    3 ... 2094 2095 2097]]: 0.9060286541791888
[mean]: 0.9212024644446475


## Тестовый датасет

In [ ]:
df_test = pd.read_csv("/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/test.csv")

In [25]:
df_test = df_test.rename(columns={"description": "desc"})
df_test.drop(["id"], axis=1, inplace=True)

In [26]:
df_test["desc"]= df_test["desc"].str.strip()
df_test["desc_len"] = df_test["desc"].str.len()

df_test["desc_1"] = df_test["desc"].apply(defineBaseDesc)
df_test["desc_1"] = df_test["desc_1"].apply(desc1CheckUp)
df_test = pd.get_dummies(df_test, columns=["desc_1"], dtype='int8')

In [ ]:
df_test

,desc,desc_len,desc_1_Алебарда,desc_1_Альбом фотографий,desc_1_Бердыш,desc_1_Бомбомет,desc_1_Винтовка,desc_1_Гербарий,desc_1_Граната,desc_1_Две,...,desc_1_Фотонегатив пленочный,desc_1_Фотонегатив стеклянный,desc_1_Фотооткрытка,desc_1_Часы,desc_1_Шашка,desc_1_Штык,desc_1_Эскиз,desc_1_Этюд,desc_1_Ядро,desc_1_Ятаган
0,Фотография. Елизавета Алексеевна Юманова. ПКМ...,176,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Фотография. Заседание комитета комсомола мотор...,466,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо...",215,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Фотография. Елизавета Алексеевна Юманова. ПКМ...,176,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Фотография. ""Универмаг Пермь. Оформление демон...",194,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Фотонегатив пленочный. Делегаты VI съезда комп...,221,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
896,"Изразец печной ПОКМ-20318/52 формовка,глина (...",54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
897,"Фотография. ""Универмаг Пермь. Оформление демон...",356,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
898,ПЗС. Подвеска шумящая коньковая ПКМ-21222/274...,548,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# LANG MODELS

langModels = ["cointegrated/rubert-tiny2", "cointegrated/LaBSE-en-ru", "sberbank-ai/ruBert-base"]
# "cointegrated/LaBSE-en-ru" - ??? use it on not ???
for model in langModels:
    df_test = df_test.join(getEmbeddings(df_test, model, "desc")) # It's necessary to check 64, 32 maxLen
df_test.drop(["desc"], axis=1, inplace=True)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 900/900 [00:20<00:00, 43.17it/s]
Some weights of the model checkpoint at cointegrated/LaBSE-en-ru 

In [ ]:
df_test.head(3)

,desc_len,desc_1_Алебарда,desc_1_Альбом фотографий,desc_1_Бердыш,desc_1_Бомбомет,desc_1_Винтовка,desc_1_Гербарий,desc_1_Граната,desc_1_Две,desc_1_Диапозитив стеклянный,...,ruBert-base_f758,ruBert-base_f759,ruBert-base_f760,ruBert-base_f761,ruBert-base_f762,ruBert-base_f763,ruBert-base_f764,ruBert-base_f765,ruBert-base_f766,ruBert-base_f767
0,176,0,0,0,0,0,0,0,0,0,...,-74.628784,-33.511482,-59.709686,32.313740,-30.018372,-54.542648,13.464134,34.649918,-9.921524,8.044344
1,466,0,0,0,0,0,0,0,0,0,...,15.092916,-45.753708,-17.130079,33.351154,-52.050606,-38.068573,-4.157882,34.151184,48.378727,38.208828
2,215,0,0,0,0,0,0,0,0,0,...,-82.179054,-42.995476,-34.295940,44.042896,-46.232422,-51.086960,12.416584,21.543226,-9.793234,6.885200


In [ ]:
#df_test.to_csv('/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/test_tokens.csv', index=False )

In [124]:
#df_test = pd.read_csv("/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/test_tokens.csv")

In [125]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [126]:
df_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df_test.columns.values]

In [127]:
train_features = list(train)
for col in train_features:
  if col not in list(df_test):
    df_test[col] = 0
test = df_test[train_features]

In [40]:
test

,object_img,len_description,object,desc_len,path,desc_1_Авиабомба,desc_1_Алебарда,desc_1_Альбом,desc_1_Амбары,desc_1_Бердыш,...,ruBert-base_f758,ruBert-base_f759,ruBert-base_f760,ruBert-base_f761,ruBert-base_f762,ruBert-base_f763,ruBert-base_f764,ruBert-base_f765,ruBert-base_f766,ruBert-base_f767
0,0,0,0,176,0,0,0,0,0,0,...,-74.628784,-33.511482,-59.709686,32.313740,-30.018372,-54.542650,13.464134,34.649918,-9.921524,8.044344
1,0,0,0,466,0,0,0,0,0,0,...,15.092916,-45.753708,-17.130080,33.351154,-52.050606,-38.068573,-4.157882,34.151184,48.378727,38.208828
2,0,0,0,215,0,0,0,0,0,0,...,-82.179054,-42.995476,-34.295940,44.042896,-46.232420,-51.086960,12.416584,21.543226,-9.793234,6.885199
3,0,0,0,176,0,0,0,0,0,0,...,-73.867620,-34.072380,-61.150253,31.845257,-28.896402,-54.286663,14.236713,34.000126,-10.674456,7.525657
4,0,0,0,194,0,0,0,0,0,0,...,-84.346290,-33.033060,-51.950886,31.934925,-35.042816,-44.474790,14.491196,20.436275,-7.008326,-4.773473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0,0,0,221,0,0,0,0,0,0,...,-60.322230,-33.771230,-51.495163,44.152607,-18.534502,-54.737106,17.684591,13.056156,-10.237722,14.954814
896,0,0,0,54,0,0,0,0,0,0,...,-106.195724,-42.484207,-80.080475,65.148346,-46.347588,-58.716220,7.492046,37.409115,-33.784714,-22.019184
897,0,0,0,356,0,0,0,0,0,0,...,-51.904250,-21.248138,-26.960443,13.636317,-34.868607,-35.863490,15.255716,31.916702,11.322696,10.063649
898,0,0,0,548,0,0,0,0,0,0,...,-32.093160,1.317588,-20.604767,43.007507,-4.887054,-20.025198,11.621331,2.338999,56.979790,48.612400


In [128]:
test.drop(columns =  ['object_img', 'desc_len', 'path'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [129]:
pred = model.predict(test).astype(int)

# Сохранение результатов

In [130]:
def save_res(dataset):
  df = pd.DataFrame(data=pred, columns = ['object_img'])
  dataset.drop(columns = 'object_img', inplace = True)
  dataset = sample.join(df)
  return dataset

In [131]:
for i in sample.object_img:
  if i in sample.object_img[sample.object_img.duplicated()]:
    sample.object_img.replace(i, randint(i-2, i+2), inplace = True)
  if i<0:
    sample.object_img.replace(i, abs(i), inplace = True)

In [132]:
sample = save_res(sample)

In [133]:
sample.to_csv('/disk/MyDrive/Colab_Notebooks/test_dataset_test_1/my_submission.csv', index=False )